In [1]:
import re
import boto3
import json
import dask
import dask.dataframe as dd
from dask.distributed import Client, progress
client = Client(n_workers=8, threads_per_worker=2, memory_limit='2GB')
client

Client Scheduler: tcp://127.0.0.1:43205 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 16 Memory: 16.00 GB


In [2]:
AVSLS_BUCKET = 'miba_ma_prj_aviasales'
with open('access.json') as file:
    access_data = json.load(file)

In [3]:
session = boto3.session.Session()
s3 = session.client(
    service_name='s3',
    aws_access_key_id=access_data['aws_access_key_id'],
    aws_secret_access_key=access_data['aws_secret_access_key'],
    endpoint_url='https://hb.bizmrg.com'
)
all_files = [key['Key'] for key in s3.list_objects(Bucket=AVSLS_BUCKET)['Contents']]
all_paths = [f's3://{AVSLS_BUCKET}/{path}' for path in all_files if '.csv' in path]
print(all_paths[:10])

['s3://miba_ma_prj_aviasales/backlinks/2lka.net.csv', 's3://miba_ma_prj_aviasales/backlinks/7eer.net.csv', 's3://miba_ma_prj_aviasales/backlinks/Tradedoubler.com.csv', 's3://miba_ma_prj_aviasales/backlinks/ab-in-den-urlaub.de.csv', 's3://miba_ma_prj_aviasales/backlinks/admitad.com.csv', 's3://miba_ma_prj_aviasales/backlinks/aegeanair.com.csv', 's3://miba_ma_prj_aviasales/backlinks/affilired.com.csv', 's3://miba_ma_prj_aviasales/backlinks/agoda.com.csv', 's3://miba_ma_prj_aviasales/backlinks/aida.de.csv', 's3://miba_ma_prj_aviasales/backlinks/airbnb.ae.csv']


In [4]:
len(all_paths)

167

In [5]:
ddf = dd.read_csv(
    all_paths[:10],
    storage_options={
        'key': access_data['aws_access_key_id'],
        'secret': access_data['aws_secret_access_key'],
        'client_kwargs': {
            'endpoint_url': 'https://hb.bizmrg.com',
        }
    },
    assume_missing=True
)
ddf.describe()

,ahrefs_rank,domain_rating,ahrefs_top,links_internal,links_external,page_size,deleted_at,refdomains,linked_root_domains,positions,traffic,redirect_code,del_reason,lost_redirect_reason,lost_redirect_source,lost_redirect_new_target,drop_reason,http_code,total_backlinks
npartitions=1,,,,,,,,,,,,,,,,,,,
,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [6]:
ddf.columns

Index(['url_from', 'refdomain', 'ahrefs_rank', 'domain_rating', 'ahrefs_top',
       'ip_from', 'links_internal', 'links_external', 'page_size', 'encoding',
       'title', 'language', 'url_to', 'first_seen', 'last_visited',
       'prev_visited', 'deleted_at', 'original', 'content', 'refdomains',
       'linked_root_domains', 'positions', 'traffic', 'redirect_code', 'alt',
       'anchor', 'del_reason', 'lost_redirect_reason', 'lost_redirect_source',
       'lost_redirect_new_target', 'drop_reason', 'in_raw', 'in_rendered',
       'text_pre', 'text_post', 'redirect_chain', 'http_code',
       'url_from_first_seen', 'first_origin', 'last_origin', 'total_backlinks',
       'powered_by', 'all', 'text', 'image', 'nofollow', 'ugc', 'sponsored',
       'dofollow', 'redirect', 'canonical', 'gov', 'edu', 'rss', 'alternate'],
      dtype='object')

In [7]:
%%time
ddf.groupby('language').url_from.count().compute()

CPU times: user 5.59 s, sys: 1.3 s, total: 6.89 s
Wall time: 41.2 s


language
bg         16230
da         15976
de         85847
en       1443061
fi          1973
fr        245257
id         14792
it         75906
ko         44047
nl         13959
pl         78508
pt         45786
ru        798627
uk          1701
cs          1013
es         57079
et         17689
he          2133
hr           330
hu          1616
ja          6082
lt         24419
ro         11312
sk            43
sl           265
sv         78138
ta            18
tl          1680
tr          2408
ur          3663
zh-cn      11141
zh-tw      36974
bn            98
el          7858
gu            14
mr            27
th         17348
vi          6089
kn             9
so          2920
ar          4367
pa            24
mk          8782
hi          4819
no           238
fa           118
lv           194
ne            15
sw            12
af             8
te            23
ml            19
sq            10
Name: url_from, dtype: int64

In [13]:
ddf = dd.read_csv(
    's3://miba-ma-prj-aviasales/backlinkslarge/tc.tradetracker.net',
    storage_options={
        'key': access_data['aws_access_key_id'],
        'secret': access_data['aws_secret_access_key'],
        'client_kwargs': {
            'endpoint_url': 'https://hb.bizmrg.com',
        }
    },
    assume_missing=True
)
ddf.describe()

,ahrefs_rank,domain_rating,ahrefs_top,links_internal,links_external,page_size,deleted_at,refdomains,linked_root_domains,positions,traffic,redirect_code,del_reason,lost_redirect_reason,lost_redirect_source,lost_redirect_new_target,drop_reason,http_code,total_backlinks
npartitions=1,,,,,,,,,,,,,,,,,,,
,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [17]:
def find_m15(url):
    url = url.lower()
    if not re.search('\?.*', url):
        return 'None'
    else:
        url = re.search('\?.*', url)
        url = url.group()
        if '?m=' in url or '&m=' in url:
            value = re.search('m=(\d{1,15})', url)
            if value is not None:
                return value.group(1)
            else:
                return 'None'
        else:
            return 'None'

def find_m5(url):
    url = url.lower()
    if not re.search('\?.*', url):
        return 'None'
    else:
        url = re.search('\?.*', url)
        url = url.group()
        if '?m=' in url or '&m=' in url:
            value = re.search('m=(\d{1,5})', url)
            if value is not None:
                return value.group(1)
            else:
                return 'None'
        else:
            return 'None'

In [18]:
ddf = ddf.assign(
    find_m5_col=ddf.url_to.map(lambda x: find_m5(x), meta=('x', str)),
    find_m15_col=ddf.url_to.map(lambda x: find_m15(x), meta=('x', str))
)

In [19]:
%%time
ddf.compute()

CPU times: user 27.1 s, sys: 30.3 s, total: 57.4 s
Wall time: 1min 49s


,url_from,refdomain,ahrefs_rank,domain_rating,ahrefs_top,ip_from,links_internal,links_external,page_size,encoding,...,sponsored,dofollow,redirect,canonical,gov,edu,rss,alternate,find_m5_col,find_m15_col
0,https://www.de-koopjeshoek.nl/bosck-3d-design-...,de-koopjeshoek.nl,0.0,0.0,0.0,104.31.66.210,8.0,1.0,4312.0,utf8,...,False,False,False,False,False,False,False,False,56544,565442
1,https://www.q-reisen.de/lopesan-und-ifa-hotels...,q-reisen.de,0.0,0.0,0.0,109.70.6.239,29.0,5.0,7800.0,utf8,...,False,False,False,False,False,False,False,False,25477,254775
2,http://www.vakantiehuizentotaal.nl/search.php?...,vakantiehuizentotaal.nl,0.0,0.0,147442075.0,83.98.209.30,76.0,25.0,6953.0,utf8,...,False,False,False,False,False,False,False,False,25586,255865
3,https://dire.fi/polkupyorat-2/t/naisten-kaupun...,dire.fi,0.0,0.0,99828147.0,51.255.135.35,31.0,4.0,11107.0,utf8,...,False,True,False,False,False,False,False,False,25571,255714
4,https://zadenplanten.nl/product/sunflower-tall...,zadenplanten.nl,0.0,0.0,129997505.0,91.184.0.6,113.0,11.0,284286.0,utf8,...,False,False,False,False,False,False,False,False,28092,280924
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37992,https://www.visitnorway.com/,visitnorway.com,84.0,79.0,15574.0,192.229.220.65,223.0,89.0,209037.0,utf8,...,False,True,False,False,False,False,False,False,12,12
37993,https://www.visitnorway.com/,visitnorway.com,84.0,79.0,15574.0,192.229.220.65,223.0,89.0,209037.0,utf8,...,False,True,False,False,False,False,False,False,11398,1139815
37994,https://www.visitnorway.com/,visitnorway.com,84.0,79.0,15574.0,192.229.220.65,223.0,89.0,209037.0,utf8,...,False,True,False,False,False,False,False,False,12,12
37995,https://www.visitnorway.com/,visitnorway.com,84.0,79.0,15574.0,192.229.220.65,223.0,89.0,209037.0,utf8,...,False,True,False,False,False,False,False,False,12,12
